### Basic working of Google Palm LLM in LangChain

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA

api_key = 'YOUR_API_KEY'

llm = GoogleGenerativeAI(model="models/text-bison-001"google_api_key=api_key, temperature=0.6)

### Now let's load data from the movie qoutes csv file

In [ ]:
loader = CSVLoader(file_path="movie_quotes.csv")

# Store the loaded data in the 'data' variable
data = loader.load()

### Hugging Face Embeddings

In [ ]:
# it requires sentence-transformers
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

### Vector store using FAISS

In [ ]:
from langchain.vectorstores import FAISS
# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data, embedding=instructor_embeddings)
#vectordb = FAISS.load_local("movie_faiss_index",embeddings=instructor_embeddings, allow_dangerous_deserialization = True)
# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
vectordb.save_local("movie_faiss_index")

In [ ]:
rdocs = retriever.get_relevant_documents("Quote from star wars")
rdocs

As you can see above, the retriever that was created using FAISS and hugging face embedding is now capable of pulling relevant documents from our original CSV file. 

### Create RetrievalQA chain along with prompt template 🚀

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="refine",
                            retriever=retriever,
                            return_source_documents=False)

### We are all set 👍🏼 Let's ask some questions now

In [ ]:
res = chain.invoke('king kong')

In [ ]:
print(res["result"])